<a href="https://colab.research.google.com/github/jameschapman19/cca_zoo/blob/master/cca_zoo_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install cca-zoo --upgrade

Requirement already up-to-date: cca-zoo in /usr/local/lib/python3.6/dist-packages (1.1.9)


In [12]:
# Imports
import numpy as np
from cca_zoo import wrappers
from cca_zoo import data
import itertools
import os
import matplotlib.pyplot as plt
from torch.utils.data import Subset

# Load MNIST Data
os.chdir('..')
N = 1000
dataset = data.Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=True)
ids = np.arange(min(2 * N, len(dataset)))
np.random.shuffle(ids)
train_ids, val_ids = np.array_split(ids, 2)
val_dataset = Subset(dataset, val_ids)
train_dataset = Subset(dataset, train_ids)
test_dataset = data.Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=False)
test_ids = np.arange(min(N, len(test_dataset)))
np.random.shuffle(test_ids)
test_dataset = Subset(test_dataset, test_ids)
train_view_1, train_view_2, train_rotations, train_OH_labels, train_labels = train_dataset.dataset.to_numpy(
    train_dataset.indices)
val_view_1, val_view_2, val_rotations, val_OH_labels, val_labels = val_dataset.dataset.to_numpy(val_dataset.indices)
test_view_1, test_view_2, test_rotations, test_OH_labels, test_labels = test_dataset.dataset.to_numpy(
    test_dataset.indices)

# Settings

# The number of latent dimensions across models
latent_dims = 2
# The number of folds used for cross-validation/hyperparameter tuning
cv_folds = 5
# For running hyperparameter tuning in parallel (0 if not)
jobs = 2
# Number of iterations for iterative algorithms
max_iter = 10
# number of epochs for deep models
epochs = 50

# Canonical Correlation Analysis

In [13]:
"""
### Linear CCA via alternating least squares (can pass more than 2 views)
"""
linear_cca = wrappers.CCA(latent_dims=latent_dims, max_iter=max_iter)

linear_cca.fit(train_view_1, train_view_2)

linear_cca_results = np.stack(
    (linear_cca.train_correlations[0, 1], linear_cca.predict_corr(test_view_1, test_view_2)[0, 1]))

# Partial Least Squares


In [14]:
"""
### PLS with scikit-learn (only permits 2 views)
"""
pls = wrappers.PLS(latent_dims=latent_dims)

pls.fit(train_view_1, train_view_2)

pls_results = np.stack(
    (pls.train_correlations[0, 1], pls.predict_corr(test_view_1, test_view_2)[0, 1]))

# Extension to multiple views



In [15]:
"""
### (Regularized) Generalized CCA(can pass more than 2 views)
"""

gcca = wrappers.GCCA(latent_dims=latent_dims)

# small ammount of regularisation added since data is not full rank
c=[0.5,0.5,0.5]

gcca.fit(train_view_1, train_view_2,train_view_1, c=c)

gcca_results = np.stack((gcca.train_correlations[0, 1], gcca.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### (Regularized) Multiset CCA(can pass more than 2 views)
"""

mcca = wrappers.MCCA(latent_dims=latent_dims)

mcca.fit(train_view_1, train_view_2,train_view_1, c=c)

mcca_results = np.stack((mcca.train_correlations[0, 1], mcca.predict_corr(test_view_1, test_view_2)[0, 1]))

"""
### Multiset CCA by alternating least squares
"""
mcca_als = wrappers.CCA(latent_dims=latent_dims, max_iter=max_iter)

mcca_als.fit(train_view_1, train_view_2,train_view_1)

mcca_als_results = np.stack(
    (mcca_als.train_correlations[0, 1], mcca_als.predict_corr(test_view_1, test_view_2)[0, 1]))


"""
### Multiset PLS by alternating least squares
"""
mcca_pls = wrappers.PLS(latent_dims=latent_dims, max_iter=max_iter)

mcca_pls.fit(train_view_1, train_view_2,train_view_1)

mcca_pls_results = np.stack(
    (mcca_als.train_correlations[0, 1], mcca_pls.predict_corr(test_view_1, test_view_2)[0, 1]))

# Rgularised CCA solutions based on alternating minimisation/alternating least squares

We implement Witten's penalized matrix decomposition form of sparse CCA using 'pmd'

We implement Waaijenborg's penalized CCA using elastic net using 'elastic'

We implement Mai's sparse CCA using 'scca'

Furthermore, any of these methods can be extended to multiple views. Witten describes this method explicitly.

In [16]:
"""
### Sparse CCA (Penalized Matrix Decomposition) (can pass more than 2 views)
"""

# PMD
c1 = [1, 3, 7, 9]
c2 = [1, 3, 7, 9]
param_candidates = {'c': list(itertools.product(c1, c2))}

pmd = wrappers.PMD(latent_dims=latent_dims, tol=1e-5, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True, jobs=jobs,
    plot=True)

pmd_results = np.stack((pmd.train_correlations[0, 1, :], pmd.predict_corr(test_view_1, test_view_2)[0, 1, :]))

"""
### Sparse CCA (can pass more than 2 views)
"""

# Sparse CCA
c1 = [0.00001, 0.0001]
c2 = [0.00001, 0.0001]
param_candidates = {'c': list(itertools.product(c1, c2))}

scca = wrappers.SCCA(latent_dims=latent_dims, tol=1e-5, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True,
    jobs=jobs, plot=True)

scca_results = np.stack(
    (scca.train_correlations[0, 1, :], scca.predict_corr(test_view_1, test_view_2)[0, 1, :]))

"""
# Sparse CCA with ADMM
c1 = [0.00001, 0.0001]
c2 = [0.00001, 0.0001]
param_candidates = {'c': list(itertools.product(c1, c2))}

scca_admm = wrappers.SCCA_ADMM(latent_dims=latent_dims, tol=1e-5, max_iter=max_iter).gridsearch_fit(
    train_view_1,
    train_view_2,
    param_candidates=param_candidates,
    folds=cv_folds,
    verbose=True,
    jobs=jobs, plot=True)

scca_admm_results = np.stack(
    (scca_admm.train_correlations[0, 1, :], scca_admm.predict_corr(test_view_1, test_view_2)[0, 1, :]))
"""

"""
### Elastic CCA (can pass more than 2 views)
"""

# Elastic CCA
c1 = [0.001, 0.0001]
c2 = [0.001, 0.0001]
l1_1 = [0.01, 0.1]
l1_2 = [0.01, 0.1]
param_candidates = {'c': list(itertools.product(c1, c2)), 'l1_ratio': list(itertools.product(l1_1, l1_2))}

elastic = wrappers.ElasticCCA(latent_dims=latent_dims, tol=1e-5,
                              max_iter=max_iter).gridsearch_fit(train_view_1,
                                                                train_view_2,
                                                                param_candidates=param_candidates,
                                                                folds=cv_folds,
                                                                verbose=True,
                                                                jobs=jobs,
                                                                plot=True)

elastic_results = np.stack(
    (elastic.train_correlations[0, 1, :], elastic.predict_corr(test_view_1, test_view_2)[0, 1, :]))

cross validation
number of folds:  5
Best score :  1.229268604951622
{'c': (9, 9)}
cross validation
number of folds:  5
Best score :  1.276732458714244
{'c': (0.0001, 0.0001)}
cross validation
number of folds:  5
Best score :  0.12498643511537114
{'c': (0.0001, 0.0001), 'l1_ratio': (0.01, 0.1)}


# Kernel CCA

In [17]:
"""
### Kernel CCA

Similarly, we can use kernel CCA methods with [method='kernel']

We can use different kernels and their associated parameters in a similar manner to before
- regularized linear kernel CCA: parameters :  'kernel'='linear', 0<'c'<1
- polynomial kernel CCA: parameters : 'kernel'='poly', 'degree', 0<'c'<1
- gaussian rbf kernel CCA: parameters : 'kernel'='gaussian', 'sigma', 0<'c'<1
"""
# %%
# r-kernel cca
c1 = [0.9, 0.99]
c2 = [0.9, 0.99]

param_candidates = {'kernel': ['linear'], 'c': list(itertools.product(c1, c2))}

kernel_reg = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                   folds=cv_folds,
                                                                   param_candidates=param_candidates,
                                                                   verbose=True, jobs=jobs,
                                                                   plot=True)
kernel_reg_results = np.stack((
    kernel_reg.train_correlations[0, 1, :],
    kernel_reg.predict_corr(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (poly)
param_candidates = {'kernel': ['poly'], 'degree': [2, 3], 'c': list(itertools.product(c1, c2))}

kernel_poly = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                    folds=cv_folds,
                                                                    param_candidates=param_candidates,
                                                                    verbose=True, jobs=jobs,
                                                                    plot=True)

kernel_poly_results = np.stack((
    kernel_poly.train_correlations[0, 1, :],
    kernel_poly.predict_corr(test_view_1, test_view_2)[0, 1, :]))

# kernel cca (gaussian)
param_candidates = {'kernel': ['rbf'], 'sigma': [1e+1, 1e+2, 1e+3], 'c': list(itertools.product(c1, c2))}

kernel_gaussian = wrappers.KCCA(latent_dims=latent_dims).gridsearch_fit(train_view_1, train_view_2,
                                                                        folds=cv_folds,
                                                                        param_candidates=param_candidates,
                                                                        verbose=True, jobs=jobs,
                                                                        plot=True)

kernel_gaussian_results = np.stack((
    kernel_gaussian.train_correlations[0, 1, :],
    kernel_gaussian.predict_corr(test_view_1, test_view_2)[0, 1, :]))

cross validation
number of folds:  5
Best score :  1.5098116454497703
{'kernel': 'linear', 'c': (0.99, 0.9)}
cross validation
number of folds:  5
Best score :  1.0811411549353331
{'kernel': 'poly', 'degree': 3, 'c': (0.9, 0.9)}
cross validation
number of folds:  5
Best score :  1.2515382003494193
{'kernel': 'rbf', 'sigma': 100.0, 'c': (0.99, 0.9)}


# Deep CCA

DCCA can be optimized using Andrew's original tracenorm objective or Wang's DCCA by nonlinear orthogonal iterations using the argument als=True.

In [18]:
"""
### Deep Learning

We also have deep CCA methods (and autoencoder variants)
- Deep CCA (DCCA)
- Deep Canonically Correlated Autoencoders (DCCAE)

We introduce a Config class from configuration.py. This contains a number of default settings for running DCCA.

"""
from cca_zoo import deepwrapper, objectives, dcca, deep_models

# %%
# DCCA
print('DCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_model = deepwrapper.DeepWrapper(dcca_model)

dcca_model.fit(train_view_1, train_view_2, epochs=epochs)

dcca_results = np.stack((dcca_model.train_correlations[0, 1], dcca_model.predict_corr(test_view_1, test_view_2)[0, 1]))


# DCCA_NOI
# Note that als=True
print('DCCA by non-linear orthogonal iterations')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_noi_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], als=True)

dcca_noi_model = deepwrapper.DeepWrapper(dcca_noi_model)

dcca_noi_model.fit(train_view_1, train_view_2, epochs=epochs)

dcca_noi_results = np.stack(
    (dcca_noi_model.train_correlations[0, 1], dcca_noi_model.predict_corr(test_view_1, test_view_2)[0, 1]))

DCCA
total parameters:  201476
====> Epoch: 1 Average train loss: -0.2295
====> Epoch: 1 Average val loss: -0.7630
Min loss -0.76
====> Epoch: 2 Average train loss: -0.7464
====> Epoch: 2 Average val loss: -1.0584
Min loss -1.06
====> Epoch: 3 Average train loss: -1.1421
====> Epoch: 3 Average val loss: -1.1717
Min loss -1.17
====> Epoch: 4 Average train loss: -1.2775
====> Epoch: 4 Average val loss: -1.2257
Min loss -1.23
====> Epoch: 5 Average train loss: -1.3402
====> Epoch: 5 Average val loss: -1.2559
Min loss -1.26
====> Epoch: 6 Average train loss: -1.3759
====> Epoch: 6 Average val loss: -1.2751
Min loss -1.28
====> Epoch: 7 Average train loss: -1.3999
====> Epoch: 7 Average val loss: -1.2887
Min loss -1.29
====> Epoch: 8 Average train loss: -1.4186
====> Epoch: 8 Average val loss: -1.2991
Min loss -1.30
====> Epoch: 9 Average train loss: -1.4348
====> Epoch: 9 Average val loss: -1.3077
Min loss -1.31
====> Epoch: 10 Average train loss: -1.4497
====> Epoch: 10 Average val loss: 

# DGCCA and DMCCA for more than 2 views

The only change we need to make is to the objective argument to perform DGCCA and DMCCA.

In [19]:
# DGCCA
print('DGCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dgcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.GCCA)

dgcca_model = deepwrapper.DeepWrapper(dgcca_model)

dgcca_model.fit(train_view_1, train_view_2, epochs=epochs)

dgcca_results = np.stack(
    (dgcca_model.train_correlations[0, 1], dgcca_model.predict_corr(test_view_1, test_view_2)[0, 1]))

# DMCCA
print('DMCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
dmcca_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.MCCA)

dmcca_model = deepwrapper.DeepWrapper(dmcca_model)

dmcca_model.fit(train_view_1, train_view_2, epochs=epochs)

dmcca_results = np.stack(
    (dmcca_model.train_correlations[0, 1], dmcca_model.predict_corr(test_view_1, test_view_2)[0, 1]))

DGCCA
total parameters:  201476
====> Epoch: 1 Average train loss: -0.1510
====> Epoch: 1 Average val loss: -0.4711
Min loss -0.47
====> Epoch: 2 Average train loss: -0.5664
====> Epoch: 2 Average val loss: -1.1723
Min loss -1.17
====> Epoch: 3 Average train loss: -1.2070
====> Epoch: 3 Average val loss: -1.3020
Min loss -1.30
====> Epoch: 4 Average train loss: -1.3488
====> Epoch: 4 Average val loss: -1.3520
Min loss -1.35
====> Epoch: 5 Average train loss: -1.4095
====> Epoch: 5 Average val loss: -1.3759
Min loss -1.38
====> Epoch: 6 Average train loss: -1.4446
====> Epoch: 6 Average val loss: -1.3902
Min loss -1.39
====> Epoch: 7 Average train loss: -1.4712
====> Epoch: 7 Average val loss: -1.4013
Min loss -1.40
====> Epoch: 8 Average train loss: -1.4962
====> Epoch: 8 Average val loss: -1.4114
Min loss -1.41
====> Epoch: 9 Average train loss: -1.5217
====> Epoch: 9 Average val loss: -1.4208
Min loss -1.42
====> Epoch: 10 Average train loss: -1.5478
====> Epoch: 10 Average val loss:

# Deep Canonically Correlated Autoencoders
We need to add decoders in order to model deep canonically correlated autoencoders and we also use the DCCAE class which inherits from DCCA

In [20]:
from cca_zoo import dccae

# DCCAE
print('DCCAE')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784)
dccae_model = dccae.DCCAE(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dccae_model = deepwrapper.DeepWrapper(dccae_model)

dccae_model.fit(train_view_1, train_view_2, epochs=epochs)

dccae_results = np.stack(
    (dccae_model.train_correlations[0, 1], dccae_model.predict_corr(test_view_1, test_view_2)[0, 1]))

DCCAE
total parameters:  404516
====> Epoch: 1 Average train loss: 177033.7656
====> Epoch: 1 Average val loss: 155792.2969
Min loss 155792.30
====> Epoch: 2 Average train loss: 153120.7500
====> Epoch: 2 Average val loss: 134652.6250
Min loss 134652.62
====> Epoch: 3 Average train loss: 132245.5000
====> Epoch: 3 Average val loss: 115017.2344
Min loss 115017.23
====> Epoch: 4 Average train loss: 112915.2812
====> Epoch: 4 Average val loss: 99213.5312
Min loss 99213.53
====> Epoch: 5 Average train loss: 97374.1641
====> Epoch: 5 Average val loss: 86769.4297
Min loss 86769.43
====> Epoch: 6 Average train loss: 85148.4453
====> Epoch: 6 Average val loss: 75465.8047
Min loss 75465.80
====> Epoch: 7 Average train loss: 74069.2266
====> Epoch: 7 Average val loss: 65957.0000
Min loss 65957.00
====> Epoch: 8 Average train loss: 64779.8125
====> Epoch: 8 Average val loss: 58767.3281
Min loss 58767.33
====> Epoch: 9 Average train loss: 57784.1992
====> Epoch: 9 Average val loss: 53202.5781
Min 

# Deep Variational CCA

In [21]:
"""
### Deep Variational Learning
Finally we have Deep Variational CCA methods.
- Deep Variational CCA (DVCCA)
- Deep Variational CCA - private (DVVCA_p)

These are both implemented by the DVCCA class with private=True/False and both_encoders=True/False. If both_encoders,
the encoder to the shared information Q(z_shared|x) is modelled for both x_1 and x_2 whereas if both_encoders is false
it is modelled for x_1 as in the paper
"""
from cca_zoo import dvcca

# %%
# DVCCA (technically bi-DVCCA)
print('DVCCA')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
dvcca_model = dvcca.DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                          private=False)

dvcca_model = deepwrapper.DeepWrapper(dvcca_model)

dvcca_model.fit(train_view_1, train_view_2, epochs=epochs)

dvcca_model_results = np.stack(
    (dvcca_model.train_correlations[0, 1], dvcca_model.predict_corr(test_view_1, test_view_2)[0, 1]))

# DVCCA_private (technically bi-DVCCA_private)
print('DVCCA_private')
encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_1 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_2 = deep_models.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = deep_models.Decoder(latent_dims=latent_dims * 3, feature_size=784, norm_output=True)
decoder_2 = deep_models.Decoder(latent_dims=latent_dims * 3, feature_size=784, norm_output=True)
dvccap_model = dvcca.DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                           private_encoders=[private_encoder_1, private_encoder_2], private=True)

dvccap_model = deepwrapper.DeepWrapper(dvccap_model)

dvccap_model.fit(train_view_1, train_view_2, epochs=epochs)

dvccap_model_results = np.stack(
    (dvccap_model.train_correlations[0, 1], dvccap_model.predict_corr(test_view_1, test_view_2)[0, 1]))

DVCCA
total parameters:  405548
====> Epoch: 1 Average train loss: 1104.1184
====> Epoch: 1 Average val loss: 1083.6385
Min loss 1083.64
====> Epoch: 2 Average train loss: 1083.7947
====> Epoch: 2 Average val loss: 1063.4221
Min loss 1063.42
====> Epoch: 3 Average train loss: 1063.3655
====> Epoch: 3 Average val loss: 1044.7902
Min loss 1044.79
====> Epoch: 4 Average train loss: 1044.6582
====> Epoch: 4 Average val loss: 1029.1282
Min loss 1029.13
====> Epoch: 5 Average train loss: 1028.8289
====> Epoch: 5 Average val loss: 1013.7655
Min loss 1013.77
====> Epoch: 6 Average train loss: 1013.4308
====> Epoch: 6 Average val loss: 999.0323
Min loss 999.03
====> Epoch: 7 Average train loss: 998.6824
====> Epoch: 7 Average val loss: 985.9468
Min loss 985.95
====> Epoch: 8 Average train loss: 985.5276
====> Epoch: 8 Average val loss: 974.3967
Min loss 974.40
====> Epoch: 9 Average train loss: 973.8835
====> Epoch: 9 Average val loss: 963.5659
Min loss 963.57
====> Epoch: 10 Average train loss

# Convolutional Deep CCA (and using other architectures)
We provide a standard CNN encoder and decoder but users can build their own encoders and decoders by inheriting BaseEncoder and BaseDecoder for seamless integration with the pipeline

In [22]:
print('Convolutional DCCA')
encoder_1 = deep_models.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
encoder_2 = deep_models.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
dcca_conv_model = dcca.DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_conv_model = deepwrapper.DeepWrapper(dcca_conv_model)

# to change the models used change the cfg.encoder_models. We implement a CNN_Encoder and CNN_decoder as well
# as some based on brainnet architecture in cca_zoo.deep_models. Equally you could pass your own encoder/decoder models

dcca_conv_model.fit(train_view_1.reshape((-1, 1, 28, 28)), train_view_2.reshape((-1, 1, 28, 28)), epochs=epochs)

dcca_conv_results = np.stack(
    (dcca_conv_model.train_correlations[0, 1], dcca_conv_model.predict_corr(test_view_1.reshape((-1, 1, 28, 28)),
                                                                            test_view_2.reshape(
                                                                                (-1, 1, 28, 28)))[0, 1]))

Convolutional DCCA
total parameters:  2512
====> Epoch: 1 Average train loss: -0.1936
====> Epoch: 1 Average val loss: -0.3435
Min loss -0.34
====> Epoch: 2 Average train loss: -0.4168
====> Epoch: 2 Average val loss: -0.4928
Min loss -0.49
====> Epoch: 3 Average train loss: -0.6000
====> Epoch: 3 Average val loss: -0.6421
Min loss -0.64
====> Epoch: 4 Average train loss: -0.7638
====> Epoch: 4 Average val loss: -0.7862
Min loss -0.79
====> Epoch: 5 Average train loss: -0.9068
====> Epoch: 5 Average val loss: -0.9084
Min loss -0.91
====> Epoch: 6 Average train loss: -1.0224
====> Epoch: 6 Average val loss: -1.0031
Min loss -1.00
====> Epoch: 7 Average train loss: -1.1115
====> Epoch: 7 Average val loss: -1.0742
Min loss -1.07
====> Epoch: 8 Average train loss: -1.1796
====> Epoch: 8 Average val loss: -1.1283
Min loss -1.13
====> Epoch: 9 Average train loss: -1.2329
====> Epoch: 9 Average val loss: -1.1703
Min loss -1.17
====> Epoch: 10 Average train loss: -1.2756
====> Epoch: 10 Averag

# Generate Some Plots

In [23]:
"""
### Make results plot to compare methods
"""
# %%

all_results = np.stack(
    [linear_cca_results, gcca_results, mcca_results, pls_results, pmd_results, elastic_results,
     scca_results, kernel_reg_results, kernel_poly_results,
     kernel_gaussian_results, dcca_results, dgcca_results, dmcca_results, dccae_results, dvcca_model_results,
     dcca_conv_results],
    axis=0)
all_labels = ['linear', 'gcca', 'mcca', 'pls', 'pmd', 'elastic', 'scca', 'linear kernel', 'polynomial kernel',
              'gaussian kernel', 'deep CCA', 'deep generalized CCA', 'deep multiset CCA', 'deep CCAE', 'deep VCCA',
              'deep convolutional cca']

from cca_zoo import plot_utils

plot_utils.plot_results(all_results, all_labels)
plt.show()